In [17]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('..')
from pipeline import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%env "WANDB_NOTEBOOK_NAME" "mete-nn.ipynb"

env: "WANDB_NOTEBOOK_NAME"="mete-nn.ipynb"


In [6]:
pipeline = AABB245_Pipeline(num_blocks=8, extrema_th=8)
dataset_24 = PolymerDataset(['../data/AA66266AA.npy', '../data/AA66466AA.npy'], pipeline)

Processing ../data/AA66466AA.npy: 100%|██████████| 20752/20752 [01:59<00:00, 173.55it/s]


In [7]:
dataset_24.data.shape, dataset_24.labels.shape

(torch.Size([40837, 8, 19]), torch.Size([40837]))

In [8]:
train_data_24, test_data_24 = train_test_split(dataset_24)

In [9]:
model_24 = MultiOutputLSTM(input_dim=dataset_24.num_features, output_dim=dataset_24.num_classes, num_blocks=dataset_24.num_blocks)

In [16]:
train(train_dataset=train_data_24, test_dataset=test_data_24, model=model_24, batch_size=512, num_epochs=10, lr_rate=0.001)

epoch=0/10, loss=0.37271414464339614, accuracy=82.67217254638672
epoch=1/10, loss=0.372731851413846, accuracy=82.62932586669922
epoch=2/10, loss=0.37188954511657357, accuracy=82.6201400756836
epoch=3/10, loss=0.37178146466612816, accuracy=82.69054412841797
epoch=4/10, loss=0.3715766570530832, accuracy=82.67829895019531
epoch=5/10, loss=0.37162007624283433, accuracy=82.75482177734375
epoch=6/10, loss=0.3719693277962506, accuracy=82.62932586669922
epoch=7/10, loss=0.37035882798954844, accuracy=82.86195373535156
epoch=8/10, loss=0.3711086120456457, accuracy=82.72421264648438
epoch=9/10, loss=0.370423570740968, accuracy=82.74869537353516


(MultiOutputLSTM(
   (lstm): LSTM(19, 1, batch_first=True)
   (linear1): Linear(in_features=8, out_features=64, bias=True)
   (linear2): Linear(in_features=64, out_features=64, bias=True)
   (linear3): Linear(in_features=64, out_features=2, bias=True)
 ),
 {'accuracy': 0.8202522346026693,
  'f1_score': 0.8169576059850373,
  'precision': 0.8391393442622951,
  'recall': 0.7959183673469388,
  'confusion_matrix': array([[0.845, 0.155],
         [0.204, 0.796]])})

In [19]:
param_grid = {
    'data_paths': [['../data/AA66266AA.npy', '../data/AA66466AA.npy'],
                   ['../data/AA66266AA.npy', '../data/AA66566AA.npy'],
                   ['../data/AA66466AA.npy', '../data/AA66566AA.npy'],
                   ['../data/AA66266AA.npy', '../data/AA66466AA.npy', '../data/AA66566AA.npy']],
    'num_blocks': [4, 8, 16, 32, 64, 128],
    'lr_rate': [0.001, 0.01]
}

run_245(param_grid)



Processing ../data/AA66566AA.npy: 100%|██████████| 20045/20045 [00:46<00:00, 428.78it/s]


epoch=9/10, loss=0.8971244705220064, accuracy=56.660369873046875
